## Install OpenAI Library

In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


## Import Library

In [4]:
import openai
import json
import spotipy
from dotenv import dotenv_values

config = dotenv_values(".env")

## Add API key

In [5]:
openai.api_key  = config["OPENAI_API_KEY"]

## Declare Functions

In [6]:
from IPython.display import Markdown, display

def display_playlist(playlist):
    sp = spotipy.Spotify(
        auth_manager=spotipy.SpotifyOAuth(
            client_id=config["SPOTIFY_CLIENT_ID"],
            client_secret=config["SPOTIFY_CLIENT_SECRET"],
            redirect_uri="http://localhost:9999",
            scope="playlist-modify-private"
        )
    )

    current_user = sp.current_user()

    track_ids = []
    assert current_user is not None

    for item in playlist:
        artist, song = item["artist"], item["song"]
        query = f"{song} {artist}"
        search_results = sp.search(q=query, type="track", limit=10)
        track_ids.append(search_results["tracks"]["items"][0]["id"])


    created_playlist = sp.user_playlist_create(
        current_user["id"],
        public=False,
        name=args.p
    )

    sp.user_playlist_add_tracks(current_user["id"], created_playlist["id"], track_ids)

### 1st Prompt

In [7]:
def get_and_render_prompt(text):
    message=f"""
    You are a color palette generating assistant that responds to text prompt for color palettes.
    You should generate color palettes that fit the theme, mood, tone, or instructions in the prompt.
    
    Key Guidelines:
    - Avoid duplicating colors code and name within the same palette.
    - Keep the palettes within a range of 2 to 8 colors.
    - Use a JSON array to represent the color palettes, with each color code in hexadecimal format.
    
    Examples:
    - Q: Convert the following verbal description of a color palette into a list of colors: google brand
      A: ["#4285f4", "#34a853", "#fbbc05", "#ea4335"]

    - Q: Convert the following verbal description of a color palette into a list of colors: midnight pastels
      A: ["#B27DFA", "#6C5DC7", "#3E4AB0", "#2C3E8C", "#182F61", "#0B1E3E", "#080620"]

    Desired Format:
      - a JSON array of hexadecimal color codes

    Prompt: {text}
    Result:
    """

    completion = openai.Completion.create(
            model="text-davinci-003",
            prompt=message,
            max_tokens=100,
            temperature=0.8,
        )
    print(completion.choices[0].text)
    print("-----")
    print(completion)

    colors = json.loads(completion.choices[0].text.strip())
    display_colors(colors)

### 2nd Prompt (GPT-3)

In [8]:
def get_and_render_prompt(text):
    message=f"""
    Color Palette Generator

    Generate color palettes that match the given theme, mood, or instructions.

    Instructions:
    - Provide a verbal description of the color palette you want.
    - Avoid duplicating colors within the same palette.
    - Keep the palette size between 2 to 8 colors.

    Example Usage:
    - Instruction: Generate a color palette for a Google brand.
    - Output:"["#4285f4", "#34a853", "#fbbc05", "#ea4335"],["Google Blue", "Google Green", "Google Yellow", "Google Red"]"

    - Instruction: Generate a color palette for ocean pastels color palette.
    - Output:"["#83ADFC", "#53CBED", "#22D9DB", "#14D59C", "#0EC36F", "#2EAC8D", "#2A8FBC"], ["Sky Blue", "Light Blue", "Turquoise", "Mint Green", "Emerald Green", "Teal", "Navy Blue"]"

    Desired Response JSON Format:"["#color1", "#color2", ...], ["name1", "name2", ...]"

    Instruction: Generate a color palette for {text}

    Result:
    """

    completion = openai.Completion.create(
            model="text-davinci-003",
            prompt=message,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    print(completion.choices[0].text)
    print("-----")
    print(completion)

    result = completion.choices[0].text
    
    colors, names = result.split("], [")

    colors = f"""{colors}]"""
    names = f"""[{names}"""
    
    print(colors)
    print(names)
    
    colors = json.loads(colors)
    names = json.loads(names)
    
    display_colors(colors, names)

### 3rd Prompt (GPT-3.5)

In [9]:
def get_and_render_prompt_chat(text, count=7):
    messages = [
        {
            "role": "system",
            "content": """You're a helpful playlist generating assistant. 
            You should generate a list of songs and their artists according to a text prompt.
            You should return a JSON array, where each element follows this format:
            {"song": <song_title>, "artist": <artist_name>}
            """
        },
        {
            "role": "user",
            "content": "Generate a playlist of 10 songs based on this prompt: super super sad songs"
        },
        {
            "role": "assistant",
            "content": """
            [
                {"song": "Everybody Hurts", "artist": "R.E.M."},
                {"song": "Nothing Compares 2 U", "artist": "Sinead O'Connor"},
                {"song": "Tears in Heaven", "artist": "Eric Clapton"},
                {"song": "Hurt", "artist": "Johnny Cash"},
                {"song": "My Heart Will Go On", "artist": "Celine Dion"},
                {"song": "Yesterday", "artist": "The Beatles"},
                {"song": "All I Want", "artist": "Kodaline"},
                {"song": "Someone Like You", "artist": "Adele"},
                {"song": "The Sound of Silence", "artist": "Simon & Garfunkel"},
                {"song": "How to Save a Life", "artist": "The Fray"}
            ]
            """
        },
        {
            "role": "user",
            "content": f"Generate a playlist of {count} songs based on this prompt: {text}"
        },
    ]

    completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            temperature=0.7,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    
    response = completion.choices[0].message.content
    print(response)
    print("-----")
    print(completion)
    
    playlist = json.loads(response)
    
    print(playlist)
    

## Insert Your Input

In [10]:
playlist = get_and_render_prompt_chat("sweet country songs from 70s")
print(playlist)
result = display_playlist(playlist)
print(result)

            [
                {"song": "Annie's Song", "artist": "John Denver"},
                {"song": "Help Me Make It Through the Night", "artist": "Kris Kristofferson"},
                {"song": "You're My Best Friend", "artist": "Don Williams"},
                {"song": "Always on My Mind", "artist": "Willie Nelson"},
                {"song": "The Gambler", "artist": "Kenny Rogers"},
                {"song": "Stand by Your Man", "artist": "Tammy Wynette"},
                {"song": "I Will Always Love You", "artist": "Dolly Parton"}
            ]
-----
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "            [\n                {\"song\": \"Annie's Song\", \"artist\": \"John Denver\"},\n                {\"song\": \"Help Me Make It Through the Night\", \"artist\": \"Kris Kristofferson\"},\n                {\"song\": \"You're My Best Friend\", \"artist\": \"Don Williams\"},\n                {\"song\": \"Always on My M

TypeError: 'NoneType' object is not iterable